In [1]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [1]:
import configparser
from datetime import datetime
import os
import glob
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
import datetime
import pandas
import  pyspark.sql.functions as F

In [2]:
spark = SparkSession \
        .builder \
        .appName("Data lake project") \
        .getOrCreate()

In [3]:
# get filepath to song data file
filepath = "C:\\Users\\Nodebechukwu.Anyaoku\\Desktop\\NSML\\Udacity\\Data Engineering\\Data-Lakes-Project\\song_data"

In [4]:
# get all files matching extension from directory
song_data = []
for root, dirs, files in os.walk(filepath):
    files = glob.glob(os.path.join(root,'*.json'))
    for f in files :
        song_data.append(os.path.abspath(f))

In [5]:
# read song data file
dfsong = spark.read.json(song_data)


In [6]:
dfsong.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [7]:
print(dfsong.count())

71


In [8]:
dfsong.show(10)

+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|         artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|ARDR4AC1187FB371A1|           null|                    |            null|Montserrat Caball...|511.16363|        1|SOBAYLL12A8C138AF9|Sono andati? Fing...|   0|
|AREBBGV1187FB523D2|           null|         Houston, TX|            null|Mike Jones (Featu...|173.66159|        1|SOOLYAZ12A6701F4A6|Laws Patrolling (...|   0|
|ARMAC4T1187FB3FA4C|       40.82624|   Morris Plains, NJ|       -74.47995|The Dillinger Esc...|207.77751|        1|SOBBUGU12A8C13E95D|Setting Fire to S...|2004|
|ARPBNLO1187FB3D52F|       40.7145

In [9]:
filepath2 = "C:\\Users\\Nodebechukwu.Anyaoku\\Desktop\\NSML\\Udacity\\Data Engineering\\Data-Lakes-Project\\log_data"

In [51]:
# get all files matching extension from directory
log_data = []
for root, dirs, files in os.walk(filepath2):
    files = glob.glob(os.path.join(root,'*.json'))
    for f in files :
        log_data.append(os.path.abspath(f))

In [52]:


# read log data file
dflog = spark.read.json(log_data)


In [53]:
print(dflog.count())

8056


In [54]:
dflog.columns

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

In [55]:
dflog.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
3,None,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,Home,1.540872e+12,563,None,200,1542247071796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
4,None,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.541060e+12,521,None,200,1542252577796,Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20...,12


In [56]:
dflog.describe("artist").show()

+-------+------------------+
|summary|            artist|
+-------+------------------+
|  count|              6820|
|   mean|             266.5|
| stddev|109.00229355385143|
|    min|               !!!|
|    max|   ÃÂtienne Daho|
+-------+------------------+



In [57]:
dflog.select("page").dropDuplicates().sort("page").show()

+----------------+
|            page|
+----------------+
|           About|
|       Downgrade|
|           Error|
|            Help|
|            Home|
|           Login|
|          Logout|
|        NextSong|
|   Save Settings|
|        Settings|
|Submit Downgrade|
|  Submit Upgrade|
|         Upgrade|
+----------------+



In [107]:
get_timestamp = udf(lambda x: F.to_timestamp(x / 1000.0))
get_hour = udf(lambda x: F.to_timestamp(x / 1000.0). hour)
get_day =  udf(lambda x: F.to_timestamp(x / 1000.0). day)
get_week = udf(lambda x: F.to_timestamp(x / 1000.0). week)
get_year = udf(lambda x: F.to_timestamp(x / 1000.0). year)
get_month =  udf(lambda x: F.to_timestamp(x / 1000.0). month)
get_dow = udf(lambda x: F.to_timestamp(x / 1000.0). dow)

In [108]:
dflog = dflog.withColumn("start_time", get_timestamp("ts").cast("timestamp"))

In [109]:
dflog = dflog.withColumn("hour", get_hour("ts").cast("integer"))
dflog = dflog.withColumn("day", get_day("ts").cast("integer"))
dflog = dflog.withColumn("week", get_week("ts").cast("integer"))
dflog = dflog.withColumn("year", get_year("ts").cast("integer"))
dflog = dflog.withColumn("month", get_month("ts").cast("integer"))
dflog = dflog.withColumn("weekday", get_dow("ts").cast("integer"))

In [110]:
# extract columns to create time table
time_table = dflog.select(dflog['start_time'], dflog['hour'], dflog['day'], dflog['week'], dflog['month'],dflog['year'],\
                              dflog['weekday']).dropDuplicates()

In [111]:
time_table.printSchema()

root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)



In [116]:
time_table.show(5)

Py4JJavaError: An error occurred while calling o655.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 71.0 failed 1 times, most recent failure: Lost task 1.0 in stage 71.0 (TID 355) (PHC-L-0597.nlng.net executor driver): java.io.IOException: Cannot run program "python3": CreateProcess error=2, The system cannot find the file specified
	at java.lang.ProcessBuilder.start(Unknown Source)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:166)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:108)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:121)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:162)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:130)
	at org.ap

In [76]:
dfsongplay = dflog.join(dfsong, dflog.artist == dfsong.artist_name, "inner").\
select(dflog['start_time'], dflog['userId'], dflog['level'], dfsong['song_id'], dfsong['artist_id'], dflog['sessionId'],dflog['location'],dflog['userAgent'],
      ).dropDuplicates()

In [114]:
dfuser = dflog.join(dfsong,dflog.artist == dfsong.artist_name, "inner").\
select(F.to_timestamp(dflog['ts'] / 1000).alias('start_time'), dflog['userId'], dflog['level'], dfsong['song_id'], dfsong['artist_id'], dflog['sessionId'],dflog['location'],dflog['userAgent'],
      ).dropDuplicates()

In [115]:
dfuser.limit(5).toPandas()    

,start_time,userId,level,song_id,artist_id,sessionId,location,userAgent
0,2018-11-14 14:11:26.796,34,free,SOWQTQZ12A58A7B63E,ARPFHN61187FB575F6,495,"Milwaukee-Waukesha-West Allis, WI",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...
1,2018-11-14 21:16:39.796,101,free,SORRZGD12A6310DBC3,ARVBRGZ1187FB4675A,603,"New Orleans-Metairie, LA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
2,2018-11-21 22:56:47.796,15,paid,SOZCTXZ12AB0182364,AR5KOSW1187FB35FF4,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
3,2018-11-15 21:32:47.796,44,paid,SOBONFF12A6D4F84D8,ARIK43K1187B9AE54C,619,"Waterloo-Cedar Falls, IA",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...
4,2018-11-29 00:22:57.796,24,paid,SOWQTQZ12A58A7B63E,ARPFHN61187FB575F6,984,"Lake Havasu City-Kingman, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."


In [149]:
dfsong.createOrReplaceTempView("staging_songs")
dflog.createOrReplaceTempView("staging_logs")

In [152]:
songplay = spark.sql("""SELECT  DISTINCT timestamp 'epoch' + ts/1000 * interval '1 second' AS start_time, 
        user_id, 
        level,
        song_id, 
        artist_id, 
        session_id, 
        location, 
        user_agent

FROM staging_logs
JOIN staging_songs ON (staging_logs.artist = staging_songs.artist_name )
AND staging_logs.page  =  'NextSong'""") \
  .show(truncate=False)

ParseException: 
Can only use numbers in the interval value part for multiple unit value pairs interval form, but got invalid value: 1 second(line 1, pos 56)

== SQL ==
SELECT  DISTINCT timestamp 'epoch' + ts/1000 * interval '1 second' AS start_time, 
--------------------------------------------------------^^^
        user_id, 
        level,
        song_id, 
        artist_id, 
        session_id, 
        location, 
        user_agent

FROM staging_logs
JOIN staging_songs ON (staging_logs.artist = staging_songs.artist_name )
AND staging_logs.page  =  'NextSong'


In [172]:
dfsongplay.toPandas()

AttributeError: 'list' object has no attribute 'toPandas'